In [1]:
import sys, os
import h5py
import numpy as np
from datetime import datetime
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import SGD
from keras.models import Model, model_from_json, load_model
from keras.regularizers import l2
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, SpatialDropout2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
from keras.layers import BatchNormalization

from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [4]:
########################################

basedir = '.'
train_data_dir = basedir+'/train'
validation_data_dir = basedir+'/validate'
os.makedirs(basedir+'/output/models', exist_ok=True)
os.makedirs(basedir+'/output/tensorboard', exist_ok=True)
os.makedirs(basedir+'/output/checkpoints', exist_ok=True)
os.makedirs(basedir+'/output/augmented', exist_ok=True)
nb_train_samples = 1881
nb_validation_samples = 481

nb_worker = 4  #cpus for real-time image augmentation
batch_size = 32
nb_epoch = 1000
img_width, img_height = 299, 299  # target size of input (resizes pictures to this)
modelname = 'InceptionV3'

In [3]:
#################################################################
# build the ResNet50 network

base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
  #175 layers in notop
  #last layer is avg_pool
  #177 layers in full
  #last layer is flatten, then fc1000

#add our own classfication layer

x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(x)
x = Flatten(name='flatten')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax', name='predictions')(x)

#convert to model object
model = Model(input=base_model.input, output=predictions)

#add regularizers:
for layer in model.layers:
    if hasattr(layer, 'W_regularizer'):
        layer.W_regularizer = l2(l=0.001)

#Save model architechture
json_string = model.to_json()
open(basedir+'/output/models/'+modelname+'_adapted.json', 'w').write(json_string)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=SGD(lr=.01, decay=0.0002, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
from keras.models import model_from_json
json_file = open(basedir+'/output/models/'+modelname+'_adapted.json', 'r')
json_string = json_file.read()
json_file.close()
model = model_from_json(json_string)
model.load_weights(basedir+'/output/checkpoints/20161204_065740-InceptionV3_train172on-008-0.783.hdf5')

In [7]:
len(model.layers)

221

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 149, 149, 32)  896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 149, 149, 32)  128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 147, 147, 32)  9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [10]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 batchnormalization_23
47 convolution2d_

In [11]:
begintime = datetime.now().strftime("%Y%m%d_%H%M%S")

In [8]:
############ STEP ONE: train for 10 epochs

#Call backs
csvlogger = CSVLogger(basedir+'/output/'+begintime+'-'+modelname+'_traintop.csv', separator=',', append=True)
#checkpointer = ModelCheckpoint(filepath=basedir+'/output/checkpoints/'+begintime+'-'+modelname+'_traintop-{epoch:03d}-{val_acc:.3f}.hdf5', monitor='val_acc', save_weights_only=True, verbose=1, save_best_only=True)
#tensorboardlogger = TensorBoard(log_dir=basedir+'/output/tensorboard/', histogram_freq=10, write_graph=True, write_images=False)

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1.0/65535,
        rotation_range=360,
        horizontal_flip=True, vertical_flip=True, 
        fill_mode='constant', cval=0)

# this is the augmentation configuration we will use for testing: only rescaling by 16bit value range or original picture
test_datagen = ImageDataGenerator(
        rescale=1.0/65535)

train_generator = train_datagen.flow_from_directory(
        train_data_dir, target_size=(img_width, img_height), batch_size=batch_size,
        class_mode='categorical', 
        color_mode='rgb')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir, target_size=(img_width, img_height), batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb')


# train the model on the new data for a few epochs
print('Training top classifier... ')
model.fit_generator(
        train_generator, samples_per_epoch=nb_train_samples, nb_epoch=10,
        validation_data=validation_generator, nb_val_samples=nb_validation_samples, 
        callbacks=[csvlogger],
        #callbacks=[csvlogger, checkpointer, tensorboardlogger],
        nb_worker=nb_worker, pickle_safe=True)


Found 1881 images belonging to 2 classes.
Found 481 images belonging to 2 classes.
Training top classifier... 
Epoch 1/10
1856/1881 [============================>.] - ETA: 0s - loss: 0.7432 - acc: 0.5916

/usr/local/lib/python3.4/dist-packages/keras/engine/training.py:1470: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


1888/1881 [==============================] - 67s - loss: 0.7409 - acc: 0.5932 - val_loss: 0.8254 - val_acc: 0.5391
Epoch 2/10
1888/1881 [==============================] - 56s - loss: 0.7349 - acc: 0.6430 - val_loss: 1.3257 - val_acc: 0.5449
Epoch 3/10
1888/1881 [==============================] - 56s - loss: 0.6751 - acc: 0.6737 - val_loss: 1.0073 - val_acc: 0.5684
Epoch 4/10
1892/1881 [==============================] - 58s - loss: 0.6888 - acc: 0.6580 - val_loss: 0.7255 - val_acc: 0.6074
Epoch 5/10
1888/1881 [==============================] - 55s - loss: 0.6634 - acc: 0.6817 - val_loss: 0.7607 - val_acc: 0.6055
Epoch 6/10
1888/1881 [==============================] - 55s - loss: 0.6908 - acc: 0.6727 - val_loss: 0.7024 - val_acc: 0.6289
Epoch 7/10
1888/1881 [==============================] - 55s - loss: 0.6905 - acc: 0.6711 - val_loss: 0.8524 - val_acc: 0.6113
Epoch 8/10
1892/1881 [==============================] - 56s - loss: 0.6793 - acc: 0.6707 - val_loss: 0.7576 - val_acc: 0.6230
Epo

In [11]:
model.save_weights(modelname+'_trainedtop.hdf5')

```
+------------------------------------------------------+                       
| NVIDIA-SMI 352.99     Driver Version: 352.99         |                       
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 0000:00:1E.0     Off |                    0 |
| N/A   62C    P0    97W / 149W |  11018MiB / 11519MiB |     51%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID  Type  Process name                               Usage      |
|=============================================================================|
|    0      1747    C   /usr/bin/python3                             10961MiB |
+-----------------------------------------------------------------------------+
```

In [12]:
from keras.models import model_from_json, load_model
json_file = open(basedir+'/output/models/'+modelname+'_adapted.json', 'r')
json_string = json_file.read()
json_file.close()
model = model_from_json(json_string)
model.load_weights(basedir+'/output/checkpoints/20161204_065740-InceptionV3_train172on-008-0.783.hdf5')

In [ ]:
############ STEP two: train for 1000 epochs

begintime = datetime.now().strftime("%Y%m%d_%H%M%S")
### Open up last convolution block 5 for training
for layer in model.layers[:172]:
    layer.trainable = False
for layer in model.layers[172:]:
    layer.trainable = True

#compile to take effect
model.compile(optimizer=SGD(lr=.001, decay=0.0002, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


#Call backs
csvlogger = CSVLogger(basedir+'/output/'+begintime+'-'+modelname+'_train172on.csv', separator=',', append=True)
checkpointer = ModelCheckpoint(filepath=basedir+'/output/checkpoints/'+begintime+'-'+modelname+'_train172on-{epoch:03d}-{val_acc:.3f}.hdf5', monitor='val_acc', save_weights_only=True, verbose=1, save_best_only=True)
tensorboardlogger = TensorBoard(log_dir=basedir+'/output/tensorboard/', histogram_freq=0, write_graph=True, write_images=False)


# train
print('Training top classifier... ')
model.fit_generator(
        train_generator, samples_per_epoch=nb_train_samples, nb_epoch=nb_epoch,
        validation_data=validation_generator, nb_val_samples=nb_validation_samples, 
        #callbacks=[csvlogger],
        callbacks=[csvlogger, checkpointer, tensorboardlogger],
        nb_worker=nb_worker, pickle_safe=True)
model.save(modelname+'_trained172on.hdf5')